In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from numpy import genfromtxt
import joblib

In [2]:
tf.__version__

'2.7.0'

In [3]:
python.__version__

NameError: name 'python' is not defined

load data

In [2]:
user_vecs = genfromtxt('./data/user_vecs.csv', delimiter=',')
item_vecs = genfromtxt('./data/item_vecs.csv', delimiter=',')
item_embeddings = genfromtxt('./data/item_embeddings.csv', delimiter=',')

In [3]:
vms = item_embeddings

load scaler

In [4]:
scalerUser = joblib.load('scalerUser.save')
scalerItem = joblib.load('scalerItem.save')
scalerTarget = joblib.load('scalerTarget.save')

Create dictionary for user_feature

In [5]:
user_dict = {}
for i in range(len(user_vecs)):
    user_dict[int(user_vecs[i][0])] = user_vecs[i]

load model

In [6]:
model = tf.keras.models.load_model('user_embedding_model')

2023-05-11 15:44:19.116632: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Generate user embedding

In [ ]:
uid = 7

In [ ]:
user_vec = user_dict[uid].reshape(1,17)

In [ ]:
scaled_user_vec = scalerUser.transform(user_vec)
vu = model.predict(scaled_user_vec[:,3:])

In [ ]:
y_p = np.dot(vms, vu.T)
y_pu = scalerTarget.inverse_transform(y_p)

In [ ]:
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first

Return the top 10 movie IDs

In [ ]:
item_vecs[sorted_index[:10],0].astype(int)

In [7]:
def retrieve(uid):
    #compute user embedding
    user_vec = user_dict[uid].reshape(1,17)
    scaled_user_vec = scalerUser.transform(user_vec)
    vu = model.predict(scaled_user_vec[:,3:])
    
    #compute dot product between user embedding and item embedding
    y_p = np.dot(vms, vu.T)
    y_pu = scalerTarget.inverse_transform(y_p)
    
    #Sort the result
    sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
    
    #return the top 10 movies
    result = item_vecs[sorted_index[:10],0].astype(int)
    
    return result
    

deploy a fastapi

In [8]:
import os
import io
import uvicorn
import numpy as np
import json
import nest_asyncio
from enum import Enum
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse

In [9]:
from pydantic import BaseModel

In [10]:
app = FastAPI(title='Deploying a ML Model with FastAPI')

In [11]:
class User(BaseModel):
    uid: int

In [16]:
@app.get("/")
def home():
    return "Congratulations! Your API is working as expected. Now head over to http://localhost:8000/docs."

@app.post("/predict") 
def prediction(user: User):

    payload = user.uid
    print('payload is:',payload)
    response = retrieve(payload).tolist()
    print("response is:", response)
    
    return {"Recommended":response}

In [ ]:
# Allows the server to be run in this interactive environment
nest_asyncio.apply()

# Host depends on the setup you selected (docker or virtual env)
host = "0.0.0.0" if os.getenv("DOCKER-SETUP") else "127.0.0.1"

# Spin up the server!    
uvicorn.run(app, host=host, port=8000)

INFO:     Started server process [79761]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51462 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51462 - "GET /openapi.json HTTP/1.1" 200 OK
payload is: 2
response is: [150548, 114935, 105504, 80906, 115149, 81562, 27846, 122920, 104841, 96610]
INFO:     127.0.0.1:51464 - "POST /predict HTTP/1.1" 200 OK
